In [ ]:
import os
import numpy as np
from scipy.stats import norm

# %matplotlib inline
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
import matplotlib.pyplot as plt

from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.datasets import MapDatasetOnOff, Datasets
from gammapy.estimators import ExcessMapEstimator
from gammapy.makers import RingBackgroundMaker

from gammapy.data import DataStore
from gammapy.maps import MapAxis
from gammapy.datasets import MapDataset
from gammapy.makers import MapDatasetMaker, SafeMaskMaker

from gammapy.stats import WStatCountsStatistic

In [ ]:
# This is directory with IRFs which were used to produce DL3 files
irf_dir = ".../IRFs/"

# This is directory containing hdu-index.fits.gz and obs-index.fits.gz. These can be either for one particular 
# night (as authomaticaly produced in Daily analysis), or for any other time interval. One can use 
# https://github.com/SST-1M-collaboration/sst1mpipe/blob/main/sst1mpipe/scripts/create_hdu_indexes.py
# script to produce joint HDU indexes for some bunch of data to be analyzed.
datastore = ""

# Here you should specify some coordinates where you expect to see something interesting.
# Either this way:
target_position = SkyCoord.from_name("source name")

# Or this way:
# source_pos = SkyCoord(10., -25, frame="icrs", unit="deg")
# See https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html

In [ ]:
os.environ['CALDB'] = irf_dir

In [ ]:
# load datastore
data_store = DataStore.from_dir(datastore)
data_store.obs_table.sort('TSTART')

In [ ]:
# We can remove some obsids (i.e. wobbles) we do not like
# These particular ones are just examples
bad_obsids = np.array([202309140409, 202309170461, 202310110480])
obsid_mask = [obsid not in list(bad_obsids.astype(int)) for obsid in data_store.obs_table["OBS_ID"]]
good_obs_list = data_store.obs_table[obsid_mask]['OBS_ID']

In [ ]:
data_store.obs_table[obsid_mask]

In [ ]:
observations = data_store.get_observations(good_obs_list)

# Theta2 plot

In [ ]:
#This is how you can plot theta2 plot directly from the final DL3 files

In [ ]:
theta2_axis = MapAxis.from_bounds(0, 0.5, nbin=30, interp="lin", unit="deg2")
norm_theta = [0.5, 0.7] * u.deg
n_off = 5
theta_cut = 0.2 * u.deg

theta2_off = np.zeros([len(theta2_axis.edges)-1, n_off])
off_radec = []
counts_all_on = []
counts_all_all_off=[]

sum_norm_on = 0
sum_norm_off = 0
N_on = 0
N_off = 0
t_elapsed = 0
rate_off_all_all = []
n_off_all_all = []

for observation in observations:
    
    mask = data_store.obs_table['OBS_ID'] == observation.obs_id
    t_elapsed += data_store.obs_table[mask]['LIVETIME']

    # ON counts
    separation = source_pos.separation(observation.events.radec)
    
    N_on += sum(separation < theta_cut)
    
    counts_on, _ = np.histogram(separation ** 2, bins = theta2_axis.edges)
    counts_all_on.append(counts_on)
    
    norm_on = (separation > norm_theta[0]) & (separation < norm_theta[1])
    sum_norm_on += sum(norm_on)

    # OFF counts
    pos_angle = observation.pointing_radec.position_angle(source_pos)
    sep_angle = observation.pointing_radec.separation(source_pos)

    # Calculate the OFF counts from the wobble positions (OFF regions) provided
    rotation_step = 360 / (n_off + 1)
    rotations_off = np.arange(0, 359, rotation_step) * u.deg
    rotations_off = rotations_off[rotations_off.to_value("deg") != 0]
    rotations_off = pos_angle + rotations_off
    
    #sum_norm_off = 0
    counts_all_off = []
    for i_off, rotation in enumerate(rotations_off, start=0):
        position_off = observation.pointing_radec.directional_offset_by(rotation, sep_angle)

        separation_off = position_off.separation(observation.events.radec)
        
        N_off += sum(separation_off < theta_cut)
        
        counts_off_wob, _ = np.histogram(separation_off ** 2, bins = theta2_axis.edges)
        
        norm_off = (separation_off > norm_theta[0]) & (separation_off < norm_theta[1])
        sum_norm_off += sum(norm_off)

        counts_all_off.append(counts_off_wob)

    #alpha = sum_norm_on/sum_norm_off
    counts_all_all_off.append(np.sum(np.array(counts_all_off), axis=0))

alpha = sum_norm_on/sum_norm_off

stat = WStatCountsStatistic(n_on=N_on, n_off=N_off, alpha=alpha)
significance_lima = stat.sqrt_ts
N_excess = N_on - alpha*N_off

counts_on=np.sum(counts_all_on, axis=0)
counts_off=np.sum(np.array(counts_all_all_off), axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
ax.errorbar(theta2_axis.center, counts_on, yerr=np.sqrt(counts_on), fmt='o', ms=10)
ax.errorbar(theta2_axis.center, alpha*counts_off, yerr=alpha*np.sqrt(counts_off), fmt='o', ms=10)
ax.set_xlabel("$\\theta^{2} [deg^{2}]$")
ax.set_ylabel("Counts")
ax.grid(ls='dashed')
ax.axvline(theta_cut.to_value()**2, color='black',ls='--',alpha=0.75)
ax.set_xlim(theta2_axis.bounds[0].value, theta2_axis.bounds[1].value)

textstr = r'N$_{{\rm on}}$ = {:.0f} '\
            f'\n'\
            r'N$_{{\rm off}}$ = {:.0f} '\
            f'\n'\
            r'N$_{{\rm excess}}$ = {:.0f} '\
            f'\n'\
            r'n$_{{\rm off \, regions}}$ = {:.0f} '\
            f'\n'\
            r'Time = {:.1f}'\
            f'\n'\
            r'LiMa Significance = {:.1f} $\sigma$ '.format(N_on,
                                                      N_off,
                                                      N_excess,
                                                      n_off,
                                                      t_elapsed.to(u.h)[0],
                                                      significance_lima)

props = dict(boxstyle='round', facecolor='wheat', alpha=0.95)
txt = ax.text(0.50, 0.96, textstr, transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)


# Skymaps

In [ ]:
# This settings follows the gammapy examples

config = AnalysisConfig()
# Select observations - 5 degrees from the source position (in case your indexes indexing all data collected by the telescope for example)
config.observations.datastore = datastore
config.observations.obs_cone = {
    "frame": "icrs",
    "lon": source_pos.ra,
    "lat": source_pos.dec,
    "radius": 5 * u.deg,
}

config.datasets.type = "3d"
config.datasets.geom.wcs.skydir = {
    "lon": source_pos.ra,
    "lat": source_pos.dec,
    "frame": "icrs",
}  
config.datasets.geom.wcs.width = {"width": "5 deg", "height": "5 deg"}
config.datasets.geom.wcs.binsize = "0.02 deg"

# Cutout size (for the run-wise event selection)
config.datasets.geom.selection.offset_max = 5 * u.deg

# We now fix the energy axis for the counts map - (the reconstructed energy binning)
config.datasets.geom.axes.energy.min = "1 TeV"
config.datasets.geom.axes.energy.max = "100 TeV"
config.datasets.geom.axes.energy.nbins = 20

# We need to extract the ring for each observation separately, hence, no stacking at this stage
config.datasets.stack = False

# safe masks
config.datasets.safe_mask.methods = ["aeff-max", "edisp-bias"]
config.datasets.safe_mask.parameters = {"aeff_percent": 1, "bias_percent": 30}

In [ ]:
config.observations.obs_ids = list(np.array(good_obs_list))

In [ ]:
analysis = Analysis(config)

# for this specific case,w e do not need fine bins in true energy
analysis.config.datasets.geom.axes.energy_true = (
    analysis.config.datasets.geom.axes.energy
)

# `First get the required observations
analysis.get_observations()

print(analysis.config)

In [ ]:
analysis.get_datasets()

In [ ]:
# get the geom that we use
geom = analysis.datasets[0].counts.geom
energy_axis = analysis.datasets[0].counts.geom.axes["energy"]
geom_image = geom.to_image().to_cube([energy_axis.squash()])

# Make the exclusion mask
region_source = CircleSkyRegion(center=source_pos, radius=0.2 * u.deg)
exclusion_mask = ~geom_image.region_mask([region_source])

# Or you can mask even more regions to prevent biased bkg estimations, if you know there is another gamma-ray source nearby
#source_pos2 = SkyCoord(ra=49.94999*u.degree, dec=41.511666*u.degree, frame='icrs')
#region_source2 = CircleSkyRegion(center=source_pos2, radius=0.2 * u.deg)
#exclusion_mask = ~geom_image.region_mask([region_source, region_source2])

ax = exclusion_mask.sum_over_axes().plot()
plt.show()

In [ ]:
# For skymaps we estimate background from a ring around each position withing the FoV
ring_maker = RingBackgroundMaker(
    r_in="0.6 deg", width="0.2 deg", exclusion_mask=exclusion_mask
)

In [ ]:
energy_axis_true = analysis.datasets[0].exposure.geom.axes["energy_true"]
stacked_on_off = MapDatasetOnOff.create(
    geom=geom_image, energy_axis_true=energy_axis_true, name="stacked"
)

for dataset in analysis.datasets:
    # Ring extracting makes sense only for 2D analysis
    dataset_on_off = ring_maker.run(dataset.to_image())
    stacked_on_off.stack(dataset_on_off)

In [ ]:
print(stacked_on_off)

In [ ]:
# Using a convolution radius of 0.1 degrees
estimator = ExcessMapEstimator(0.1 * u.deg, selection_optional=[])

lima_maps = estimator.run(stacked_on_off)

significance_map = lima_maps["sqrt_ts"].copy()
excess_map = lima_maps["npred_excess"].copy()

# We can plot the excess and significance maps
fig, (ax1, ax2) = plt.subplots(
    figsize=(11, 5), subplot_kw={"projection": lima_maps.geom.wcs}, ncols=2
)

ax1.set_title("Significance map")
significance_map.plot(ax=ax1, add_cbar=True, )

ax2.set_title("Excess map")
excess_map.plot(ax=ax2, add_cbar=True)


ax1.plot(significance_map.geom.center_pix[0],
         significance_map.geom.center_pix[1],
         'x',
         color='black',
         label="Estimated source position",
        )


ax1.legend()
ax1.grid(alpha=0.4)
ax2.grid(alpha=0.4)

plt.show()

In [ ]:
# check distribution of significance for signal region and everything except the exclusion mask, i.e. the background

significance_map = lima_maps["sqrt_ts"].copy()
excess_map = lima_maps["npred_excess"].copy()

significance_map_off = significance_map * exclusion_mask
significance_all = significance_map.data[np.isfinite(significance_map.data)]
significance_off = significance_map_off.data[np.isfinite(significance_map_off.data)]

fig, ax = plt.subplots()
ax.hist(
    significance_all,
    density=True,
    alpha=0.5,
    color="red",
    label="all bins",
    bins=51,
    range=[min(significance_all), max(significance_all)]
)

ax.hist(
    significance_off,
    density=True,
    alpha=0.5,
    color="blue",
    label="off bins",
    bins=51,
    range=[min(significance_all), max(significance_all)]
)

# Now, fit the off distribution with a Gaussian
mu, std = norm.fit(significance_off)
x = np.linspace(-8, 8, 50)
p = norm.pdf(x, mu, std)
ax.plot(x, p, lw=2, color="black")
ax.legend()
ax.set_xlabel("Significance")
ax.set_yscale("log")
ax.set_ylim(1e-5, 1)
xmin, xmax = np.min(significance_all), np.max(significance_all)
ax.set_xlim(xmin, xmax)

print(f"Fit results: mu = {mu:.2f}, std = {std:.2f}")
plt.grid()
plt.show()